In [17]:
from physionet2023.modeling.scoringUtil import compute_challenge_score, roc_auc_score, regression_to_probability
import torch
from physionet2023.dataProcessing.patientDatasets import MetadataOnlyDataset
from physionet2023.dataProcessing.datasets import PatientDataset
import os
import glob
import pandas as pd
import numpy as np
metadata_ds = MetadataOnlyDataset("../data")

In [5]:
pre_df = list()

for path in glob.glob("../cache/loo/ICARE_*"):
    pid = path.split("/")[-1].split(".")[0]
    _, patient_metadata, _ = metadata_ds.get_by_pid(pid)

    pre_df.append({
        "pid": pid,
        "label": int(int(patient_metadata["CPC"]) > 2),
        'label_raw': int(patient_metadata['CPC']),
        'pred_raw': torch.load(path) * 4 + 1,
        "preds": regression_to_probability(torch.load(path) * 4 + 1),
        "Age": float(patient_metadata["Age"]),
        "Sex": patient_metadata["Sex"],
        "ROSC": float(patient_metadata["ROSC"]),
        "TTM": float(patient_metadata["TTM"]),
        "OHCA": bool(patient_metadata["OHCA"] == 'True'),
        "VFib": bool(patient_metadata["VFib"] == 'True')
    })

df = pd.DataFrame(data=pre_df)
df = df.set_index("pid")
df

,label,label_raw,pred_raw,preds,Age,Sex,ROSC,TTM,OHCA,VFib
pid,,,,,,,,,,
ICARE_0415,0,2,"[[tensor(0.8586)], [tensor(1.4531)], [tensor(1...","[[tensor(0.)], [tensor(0.1510)], [tensor(0.280...",73.0,Female,NaN,33.0,True,True
ICARE_0598,0,1,"[[tensor(2.6523)], [tensor(1.6775)], [tensor(0...","[[tensor(0.5305)], [tensor(0.2258)], [tensor(0...",51.0,Male,NaN,33.0,True,True
ICARE_0296,0,1,"[[tensor(3.4975)], [tensor(4.2839)], [tensor(2...","[[tensor(0.6995)], [tensor(0.8568)], [tensor(0...",48.0,Male,NaN,36.0,True,True
ICARE_0530,1,3,"[[tensor(2.2982)], [tensor(1.9321)], [tensor(2...","[[tensor(0.4327)], [tensor(0.3107)], [tensor(0...",57.0,Male,NaN,36.0,False,False
ICARE_0554,1,5,"[[tensor(4.3281)], [tensor(4.7304)], [tensor(3...","[[tensor(0.8656)], [tensor(0.9461)], [tensor(0...",59.0,Male,NaN,36.0,False,False
...,...,...,...,...,...,...,...,...,...,...
ICARE_0341,1,5,"[[tensor(5.5310)], [tensor(5.5923)], [tensor(5...","[[tensor(1.)], [tensor(1.)], [tensor(1.)], [te...",79.0,Male,NaN,36.0,False,False
ICARE_0365,0,1,"[[tensor(2.1085)], [tensor(2.5423)], [tensor(2...","[[tensor(0.3695)], [tensor(0.5085)], [tensor(0...",68.0,Male,NaN,33.0,True,True
ICARE_0444,1,5,"[[tensor(3.2860)], [tensor(2.8555)], [tensor(2...","[[tensor(0.6572)], [tensor(0.5711)], [tensor(0...",71.0,Male,NaN,NaN,False,False


In [6]:
df["p_mean"] = df['preds'].apply(lambda x: float(x.mean()))
df['p_median'] = df['preds'].apply(lambda x: float(x.median()))
df['p_max'] = df['preds'].apply(lambda x: float(x.max()))
df['p_min'] = df['preds'].apply(lambda x: float(x.min()))
df['p_25'] = df['preds'].apply(lambda x: float(torch.quantile(x, 0.25)))
df['p_75'] = df['preds'].apply(lambda x: float(torch.quantile(x, 0.75)))
df['p_40'] = df['preds'].apply(lambda x: float(torch.quantile(x, 0.40)))
df['p_60'] = df['preds'].apply(lambda x: float(torch.quantile(x, 0.60)))
df['p_std'] = df['preds'].apply(lambda x: float(x.std()))
df["pr_mean"] = df['pred_raw'].apply(lambda x: float(x.mean()))
df['pr_diff'] = (df['label_raw'] - df['p_median']).abs()


df

,label,label_raw,pred_raw,preds,Age,Sex,ROSC,TTM,OHCA,VFib,...,p_median,p_max,p_min,p_25,p_75,p_40,p_60,p_std,pr_mean,pr_diff
pid,,,,,,,,,,,,,,,,,,,,,
ICARE_0415,0,2,"[[tensor(0.8586)], [tensor(1.4531)], [tensor(1...","[[tensor(0.)], [tensor(0.1510)], [tensor(0.280...",73.0,Female,NaN,33.0,True,True,...,0.044907,0.373534,0.000000,0.000000,0.207968,0.016163,0.146229,0.114969,1.173380,1.955093
ICARE_0598,0,1,"[[tensor(2.6523)], [tensor(1.6775)], [tensor(0...","[[tensor(0.5305)], [tensor(0.2258)], [tensor(0...",51.0,Male,NaN,33.0,True,True,...,0.278954,0.692015,0.000000,0.156177,0.496594,0.253467,0.354753,0.200306,1.973993,0.721046
ICARE_0296,0,1,"[[tensor(3.4975)], [tensor(4.2839)], [tensor(2...","[[tensor(0.6995)], [tensor(0.8568)], [tensor(0...",48.0,Male,NaN,36.0,True,True,...,0.699490,0.897121,0.520226,0.654674,0.866858,0.730946,0.825314,0.171481,3.717010,0.300510
ICARE_0530,1,3,"[[tensor(2.2982)], [tensor(1.9321)], [tensor(2...","[[tensor(0.4327)], [tensor(0.3107)], [tensor(0...",57.0,Male,NaN,36.0,False,False,...,0.504571,0.826531,0.164325,0.360760,0.560874,0.476597,0.527032,0.144763,2.503614,2.495429
ICARE_0554,1,5,"[[tensor(4.3281)], [tensor(4.7304)], [tensor(3...","[[tensor(0.8656)], [tensor(0.9461)], [tensor(0...",59.0,Male,NaN,36.0,False,False,...,0.857080,1.000000,0.619188,0.733076,0.912507,0.827647,0.881494,0.118964,4.157576,4.142920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ICARE_0341,1,5,"[[tensor(5.5310)], [tensor(5.5923)], [tensor(5...","[[tensor(1.)], [tensor(1.)], [tensor(1.)], [te...",79.0,Male,NaN,36.0,False,False,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,5.478612,4.000000
ICARE_0365,0,1,"[[tensor(2.1085)], [tensor(2.5423)], [tensor(2...","[[tensor(0.3695)], [tensor(0.5085)], [tensor(0...",68.0,Male,NaN,33.0,True,True,...,0.381393,0.827688,0.000000,0.136829,0.521771,0.346540,0.432010,0.227182,2.104647,0.618607
ICARE_0444,1,5,"[[tensor(3.2860)], [tensor(2.8555)], [tensor(2...","[[tensor(0.6572)], [tensor(0.5711)], [tensor(0...",71.0,Male,NaN,NaN,False,False,...,0.623603,0.845498,0.490962,0.570399,0.731162,0.603892,0.669812,0.092342,3.242413,4.376397


In [7]:
def drop_outliers(preds):
    p_std = preds.std()
    p_med = preds.median()

    adjusted = torch.where(preds > (p_med + p_std), preds, p_med)
    return float(adjusted.mean())

df['p_drop_outliers'] = df['preds'].apply(drop_outliers)

df['p_drop_outliers']

pid
ICARE_0415    0.114245
ICARE_0598    0.358137
ICARE_0296    0.748898
ICARE_0530    0.521240
ICARE_0554    0.869508
                ...   
ICARE_0341    1.000000
ICARE_0365    0.409203
ICARE_0444    0.664996
ICARE_0518    0.651430
ICARE_0496    1.000000
Name: p_drop_outliers, Length: 292, dtype: float64

In [8]:
label = df['label'].to_numpy()
preds = df['p_drop_outliers'].to_numpy()

print(roc_auc_score(label, preds))
print(compute_challenge_score(label, preds))

0.7972264050128721
0.5086705202312138


In [9]:
def describe(df):
    for float_col in ["Age", "ROSC", "TTM", "p_std"]:
        print(f"{float_col} mean: {df[float_col].mean()}")
    
    for cat_col in ["Sex", "OHCA", "VFib"]:
        print(f"{cat_col} value counts:")
        print(df[cat_col].value_counts())


In [10]:
describe(df.nlargest(n=25, columns=['pr_diff']))

Age mean: 67.2
ROSC mean: 20.6875
TTM mean: 33.31578947368421
p_std mean: 0.11438935719430447
Sex value counts:
Male      17
Female     8
Name: Sex, dtype: int64
OHCA value counts:
False    13
True     12
Name: OHCA, dtype: int64
VFib value counts:
False    16
True      9
Name: VFib, dtype: int64


In [11]:
describe(df.nsmallest(n=25, columns=['pr_diff']))

Age mean: 59.5
ROSC mean: 26.0
TTM mean: 33.142857142857146
p_std mean: 0.10203122673556209
Sex value counts:
Male      16
Female     8
nan        1
Name: Sex, dtype: int64
OHCA value counts:
True     16
False     9
Name: OHCA, dtype: int64
VFib value counts:
True     16
False     9
Name: VFib, dtype: int64


In [26]:
patient_ds =  PatientDataset("../data/")

mean_stds = list()
for pid in df.nlargest(n=25, columns=['pr_diff']).index.to_list():
    _, _, recordings = patient_ds.get_by_patient_id(pid)

    mean_std = np.array([r.std() for r in recordings]).mean()
    mean_stds.append(mean_std)

print(np.array(mean_stds).mean())

15.617695
